In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/18"
pmc = "21018020" # 店舗ID
SPREADSHEET_KEY = '1-lHQ20Lj6XmvHsHqheXgE8JJz7EVB5-Kk3OcTB4aJyI' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 649 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+456@gmail.com "
user_pw = "paruparu22"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s2row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s2table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "125":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "125":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "200":
                    s2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "200":
                        s2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1.25RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S2RAW'!F" + str(int(s2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s2table})

['08/18', '025392', '1', '16154.0', '302.0', '41', '7']
['08/18', '025529', '2', '12308.0', '-3309.0', '13', '4']
['08/18', '025774', '3', '17692.0', '-7823.0', '14', '8']
['08/18', '025576', '4', '11282.0', '3612.0', '20', '4']
['08/18', '025183', '5', '8205.0', '4816.0', '13', '3']
['08/18', '025239', '6', '6667.0', '-2707.0', '6', '1']
['08/18', '025533', '7', '34872.0', '-4814.0', '37', '15']
['08/18', '025711', '8', '10769.0', '-4212.0', '7', '4']
['08/18', '025892', '9', '34359.0', '-3911.0', '36', '13']
['08/18', '025523', '10', '13846.0', '-1805.0', '18', '7']
['08/18', '025774', '11', '10256.0', '-6018.0', '6', '4']
['08/18', '025613', '12', '29487.0', '302.0', '32', '12']
['08/18', '025722', '13', '23590.0', '4816.0', '39', '9']
['08/18', '025341', '14', '3333.0', '3612.0', '11', '1']
['08/18', '025563', '15', '31538.0', '3311.0', '37', '13']
['08/18', '025759', '16', '1795.0', '-902.0', '2', '2']
['08/18', '025835', '17', '27692.0', '-11434.0', '58', '18']
['08/18', '025892'

['08/18', '025680', '143', '5385.0', '-1504.0', '2', '1']
['08/18', '025764', '144', '24103.0', '-13541.0', '5', '2']
['08/18', '025656', '145', '5897.0', '16853.0', '19', '1']
['08/18', '025606', '146', '17949.0', '-11434.0', '4', '3']
['08/18', '025794', '147', '7949.0', '-2707.0', '3', '1']
['08/18', '025550', '148', '17179.0', '-2406.0', '7', '3']
['08/18', '025856', '149', '29487.0', '-14444.0', '8', '4']
['08/18', '025606', '150', '6154.0', '302.0', '4', '1']
['08/18', '025796', '151', '23590.0', '3010.0', '18', '4']
['08/18', '025789', '152', '1282.0', '-1203.0', '0', '0']
['08/18', '025648', '153', '8718.0', '-3610.0', '7', '7']
['08/18', '025606', '154', '13333.0', '-10231.0', '1', '1']
['08/18', '025483', '155', '14615.0', '-3610.0', '9', '1']
['08/18', '025812', '156', '5897.0', '-4212.0', '1', '1']
['08/18', '025825', '157', '8462.0', '-3610.0', '4', '4']
['08/18', '025606', '158', '5897.0', '-4212.0', '1', '1']
['08/18', '025856', '159', '31795.0', '-14744.0', '9', '4']
['

['08/18', '025626', '285', '15385.0', '-2406.0', '19', '19']
['08/18', '025712', '286', '14359.0', '-3610.0', '24', '7']
['08/18', '025782', '287', '8974.0', '2107.0', '13', '2']
['08/18', '025722', '288', '12821.0', '10232.0', '35', '5']
['08/18', '025798', '289', '4359.0', '-2707.0', '1', '1']
['08/18', '025798', '290', '0', '0', '0', '0']
['08/18', '025798', '291', '0', '0', '0', '0']
['08/18', '025819', '292', '0', '0', '0', '0']
['08/18', '025819', '293', '769.0', '-1203.0', '1', '1']
['08/18', '025819', '294', '1538.0', '-902.0', '1', '1']
['08/18', '025758', '295', '0', '0', '0', '0']
['08/18', '025758', '296', '2308.0', '603.0', '7', '2']
['08/18', '025758', '297', '0', '0', '0', '0']
['08/18', '025707', '298', '3333.0', '-2707.0', '1', '1']
['08/18', '025707', '299', '0', '0', '0', '0']
['08/18', '025707', '300', '11282.0', '-1504.0', '9', '4']
['08/18', '025707', '301', '7436.0', '-6319.0', '1', '1']
['08/18', '025746', '302', '256.0', '-902.0', '0', '0']
['08/18', '025746', 

['08/18', '025867', '437', '17436.0', '-5416.0', '8', '3']
['08/18', '025867', '438', '3590.0', '10533.0', '10', '1']
['08/18', '025867', '439', '256.0', '1205.0', '1', '1']
['08/18', '025867', '440', '769.0', '-902.0', '0', '0']
['08/18', '025867', '441', '6410.0', '-2707.0', '2', '2']
['08/18', '025838', '442', '8205.0', '-6018.0', '1', '1']
['08/18', '025838', '443', '0', '0', '0', '0']
['08/18', '025838', '444', '769.0', '-1203.0', '0', '0']
['08/18', '025838', '445', '513.0', '-902.0', '0', '0']
['08/18', '025838', '446', '0', '0', '0', '0']
['08/18', '025838', '447', '4872.0', '8427.0', '9', '2']
['08/18', '025838', '448', '0', '0', '0', '0']
['08/18', '025838', '449', '0', '0', '0', '0']
['08/18', '025838', '450', '7179.0', '1205.0', '7', '3']
['08/18', '025890', '451', '4103.0', '27987.0', '39', '1']
['08/18', '025890', '452', '0', '0', '0', '0']
['08/18', '025890', '453', '1538.0', '-1504.0', '0', '0']
['08/18', '025890', '454', '7692.0', '-3610.0', '4', '2']
['08/18', '025890

['08/18', '025665', '585', '28974.0', '-7522.0', '14', '5']
['08/18', '025665', '586', '23846.0', '12038.0', '21', '3']
['08/18', '025665', '587', '25897.0', '2107.0', '17', '5']
['08/18', '025665', '588', '28462.0', '8126.0', '19', '2']
['08/18', '025665', '589', '30000.0', '25279.0', '36', '10']
['08/18', '025665', '590', '22564.0', '-16550.0', '3', '2']
['08/18', '025665', '591', '26667.0', '-10531.0', '9', '3']
['08/18', '025665', '592', '20769.0', '-16851.0', '1', '1']
['08/18', '025665', '593', '30769.0', '3311.0', '20', '4']
['08/18', '025665', '594', '35897.0', '-8726.0', '16', '5']
['08/18', '025895', '595', '36410.0', '19862.0', '35', '4']
['08/18', '025895', '596', '32308.0', '3612.0', '22', '4']
['08/18', '025895', '597', '31026.0', '-26180.0', '4', '4']
['08/18', '025895', '598', '43333.0', '-12036.0', '19', '4']
['08/18', '025895', '599', '31026.0', '-22569.0', '5', '3']
['08/18', '025895', '600', '24872.0', '-2707.0', '14', '4']
['08/18', '025895', '601', '26923.0', '201

['08/18', '120077', '721', '4938', '463.0', '0', '23', '37']
['08/18', '120077', '722', '1620', '-185.0', '0', '8', '14']
['08/18', '120077', '723', '706', '-370.0', '0', '1', '3']
['08/18', '120077', '724', '3500', '2130.0', '0', '16', '30']
['08/18', '120077', '725', '2395', '-2222.0', '0', '1', '8']
['08/18', '120093', '726', '1262', '-787.0', '5', '0', '5']
['08/18', '120093', '727', '954', '-93.0', '7', '0', '7']
['08/18', '120093', '728', '1166', '-1111.0', '3', '0', '3']
['08/18', '120093', '729', '2987', '-787.0', '17', '0', '17']
['08/18', '120108', '730', '1842', '-1065.0', '0', '0', '6']
['08/18', '120108', '731', '677', '231.0', '0', '0', '4']
['08/18', '120108', '732', '1068', '231.0', '0', '0', '8']
['08/18', '120108', '733', '2769', '-2222.0', '0', '0', '6']
['08/18', '120108', '734', '910', '-926.0', '0', '0', '1']
['08/18', '120108', '735', '4300', '1481.0', '0', '0', '35']
['08/18', '120108', '736', '9', '0', '0', '0', '--']
['08/18', '120108', '737', '992', '-833.0',

['08/18', '120122', '857', '2553', '-1019.0', '21', '9', '--']
['08/18', '120122', '858', '1735', '7917.0', '75', '4', '--']
['08/18', '120122', '859', '4478', '-1944.0', '32', '9', '--']
['08/18', '120122', '860', '1677', '1019.0', '29', '4', '--']
['08/18', '120122', '861', '5445', '4306.0', '90', '20', '--']
['08/18', '120122', '862', '4096', '-833.0', '34', '10', '--']
['08/18', '120122', '863', '4427', '-741.0', '39', '13', '--']
['08/18', '120122', '864', '6216', '-1574.0', '52', '16', '--']
['08/18', '120125', '865', '7353', '-1296.0', '21', '18', '--']
['08/18', '120125', '866', '3126', '833.0', '15', '6', '--']
['08/18', '120125', '867', '9270', '2870.0', '43', '24', '--']
['08/18', '120125', '868', '2907', '1481.0', '16', '6', '--']
['08/18', '120125', '869', '2580', '741.0', '14', '1', '--']
['08/18', '120125', '870', '5578', '-509.0', '19', '14', '--']
['08/18', '120125', '871', '1241', '-741.0', '1', '3', '--']
['08/18', '120125', '872', '304', '-370.0', '0', '0', '--']
['

['08/18', '120110', '991', '6624', '-4537.0', '23', '11', '--']
['08/18', '120110', '992', '2466', '278.0', '16', '8', '--']
['08/18', '120110', '993', '3701', '-694.0', '20', '8', '--']
['08/18', '120110', '994', '2605', '6944.0', '41', '28', '--']
['08/18', '120110', '995', '3583', '1204.0', '30', '5', '--']
['08/18', '120110', '996', '3940', '1204.0', '29', '13', '--']
['08/18', '120110', '997', '3300', '1898.0', '28', '13', '--']
['08/18', '120110', '998', '7102', '-4444.0', '22', '20', '--']
['08/18', '120110', '999', '2641', '2130.0', '28', '3', '--']
['08/18', '120110', '1000', '968', '463.0', '8', '3', '--']
['08/18', '120110', '1001', '2826', '3426.0', '32', '12', '--']
['08/18', '120110', '1002', '3150', '-1944.0', '8', '5', '--']
['08/18', '120110', '1003', '5898', '-4954.0', '11', '9', '--']
['08/18', '120110', '1004', '4380', '1343.0', '34', '13', '--']
['08/18', '120110', '1005', '2925', '-556.0', '12', '18', '--']
['08/18', '120110', '1006', '3328', '3241.0', '31', '21',

['08/18', '120094', '1129', '9', '0', '0', '0', '--']
['08/18', '120100', '1130', '1178', '-880.0', '0', '3', '7']
['08/18', '120100', '1131', '2963', '-1574.0', '0', '11', '21']
['08/18', '120069', '1132', '776', '-741.0', '0', '1', '3']
['08/18', '120069', '1133', '149', '-139.0', '0', '0', '--']
['08/18', '120076', '1134', '879', '-231.0', '0', '0', '3']
['08/18', '120076', '1135', '0', '0', '0', '0', '--']
['08/18', '120121', '1136', '1652', '-1389.0', '0', '4', '4']
['08/18', '120121', '1137', '0', '0', '0', '0', '--']
['08/18', '120121', '1138', '1385', '-648.0', '0', '1', '15']
['08/18', '120121', '1139', '970', '278.0', '0', '2', '17']
['08/18', '120126', '1140', '2896', '-2917.0', '6', '0', '6']
['08/18', '120126', '1141', '2595', '-1944.0', '7', '0', '6']
['08/18', '120126', '1142', '4243', '-1204.0', '0', '9', '20']
['08/18', '120126', '1143', '5043', '1157.0', '0', '13', '34']
['08/18', '120103', '1144', '2060', '-1019.0', '7', '2', '7']
['08/18', '120103', '1145', '988', '

{'spreadsheetId': '1-lHQ20Lj6XmvHsHqheXgE8JJz7EVB5-Kk3OcTB4aJyI',
 'updates': {'spreadsheetId': '1-lHQ20Lj6XmvHsHqheXgE8JJz7EVB5-Kk3OcTB4aJyI',
  'updatedRange': 'S2RAW!F6687:M6721',
  'updatedRows': 35,
  'updatedColumns': 8,
  'updatedCells': 280}}